<a href="https://colab.research.google.com/github/d-Revo/wtproject/blob/main/Storyblocks_Automatic1111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Storyblocks

![GitHub](https://img.shields.io/github/license/adithya-s-k/Storyblocks)
![GitHub stars](https://img.shields.io/github/stars/adithya-s-k/Storyblocks)

Created by [Adithya S K](https://twitter.com/adithya_s_k)

## Instructions

- Storyblocks uses Automatic1111 to generate the image and relies on Elevenlabs for text-to-speech. Don't worry if you don't have an Elevenlabs API key; it uses an OSS library to generate audio, but it might sound a little robotic.
- Run the first cell, and you will get the Gradio link or Ngrok link. Paste that into the second cell and witness the magic.
- Follow the `gradio.live` or `ngrok.io` link to start using AUTOMATIC1111.

This notebook launches [AUTOMATIC1111 Stable Diffusion Web UI](https://github.com/AUTOMATIC1111/stable-diffusion-webui).

## Ngrok (Optional)

Use [Ngrok](https://ngrok.com/) if you experience GUI issues like unresponsive buttons after generating images:

1. Go to [Ngrok](https://ngrok.com/).
2. Create an account.
3. Verify your email.
4. Copy the auth token from [here](https://dashboard.ngrok.com/get-started/your-authtoken) and paste it in the Ngrok field below.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os, subprocess, time, glob
import threading
#@title <font size="6" color="orange">Run Automatic1111 in one click</font>
#@markdown #### Follow the URL link **`https://xxxxxxx.gradio.live`** to launch the app.
#@markdown ##### (**`https://xxxxx.ngrok-free.app`** if use ngrok.)
output_path = 'AI_PICS' #@param {type:"string"}
NGROK ='https://9621-24-133-245-165.ngrok-free.app' #@param {type: "string"}
#@markdown Save images, settings, Lora models, embeddings, load models in Google Drive.

Save_In_Google_Drive = 'Everything' #@param ['Everything', 'Small models, images and settings (Recommended)', "Nothing"]
class GDriveSaveMode:
    Everything = 1
    Models_only = 2
    Nothing = 3
if Save_In_Google_Drive == 'Small models, images and settings (Recommended)':
  gMode = GDriveSaveMode.Models_only
elif Save_In_Google_Drive == 'Everything':
  gMode = GDriveSaveMode.Everything
elif Save_In_Google_Drive == 'Nothing':
  gMode = GDriveSaveMode.Nothing
else:
  raise ValueError("Save_In_Google_Drive value invalid.")

Clear_Log = True #@param{type:'boolean'}
v1_4_model = False
v1_5_model = False
v1_5_inpainting_model = False
F222_model = False
DreamShaper_model = False
OpenJourney_model = False
Anything_v3_model = False
Inkpunk_Diffusion_model = False
instruct_pix2pix_model = False

v2_1_768_model = False
v2_1_512_model = False
v2_depth_model = False

ControlNet = True
Deforum = True
Regional_Prompter = True
Ultimate_SD_Upscale = True
Openpose_Editor = True

#@markdown ### Install models from URL (separate them with comma).
Model_from_URL = '/content/drive/MyDrive/sd/sde/stable-diffusion-webui/models/Stable-diffusion/disneyrealcartoonmix_v10.safetensors, /content/drive/MyDrive/sd/sde/stable-diffusion-webui/models/Stable-diffusion/cutifiedanimecharact_sdxlV10.safetensors, /content/drive/MyDrive/sd/sde/stable-diffusion-webui/models/Stable-diffusion/crystalClearXL_ccxl.safetensors, /content/drive/MyDrive/sd/sde/stable-diffusion-webui/models/Stable-diffusion/bonoboXLRealistic_v20.safetensors, /content/drive/MyDrive/sd/sde/stable-diffusion-webui/models/Stable-diffusion/artUniverse_sdxlV20.safetensors, /content/drive/MyDrive/sd/sde/stable-diffusion-webui/models/Stable-diffusion/_SDXL_Turbo/Ultraspice/ultraspice_v16.safetensors, /content/drive/MyDrive/sd/sde/stable-diffusion-webui/models/Stable-diffusion/modernDisneyXL_v3.safetensors, /content/drive/MyDrive/sd/sde/stable-diffusion-webui/models/Stable-diffusion/juggernautXL_v9Rdphoto2Lightning.safetensors' #@param {type: "string"}
Save_a_copy_in_Google_Drive = True #@param{type: "boolean"}
#@markdown (Saved models will be available next time you start)

Extensions_from_URL = ''

Extra_arguments = ''

if gMode != GDriveSaveMode.Nothing:
  # connect to google drive
  from google.colab import drive
  drive.mount('/content/drive')
  output_path = '/content/drive/MyDrive/' + output_path
  if gMode == GDriveSaveMode.Everything:
    # install A1111 in AI_PIC
    root = output_path
  elif gMode == GDriveSaveMode.Models_only:
    # install A1111 in Colab
    root = '/content/'
    # make necessary directories if not exists
    !mkdir -p {output_path}/outputs
    !mkdir -p {output_path}/models
    !mkdir -p {output_path}/ESRGAN
    !mkdir -p {output_path}/hypernetworks
  else:
    raise ValueError("Unexpected gMode: %s"%gMode)
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path
  root = '/content/'


def clear():
    from IPython.display import clear_output; return clear_output()

def fetch_bytes(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        from urllib.request import urlopen
        return urlopen(url_or_path)
    return open(url_or_path, 'r')

def packages():
    import sys, subprocess
    return [r.decode().split('==')[0] for r in subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).split()]

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

def download_models():
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  print('⏳ Downloading models ...')
  if v1_4_model:
    downloadModel('https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt')
  if v1_5_model:
    downloadModel('https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt')
  if v1_5_inpainting_model:
    downloadModel('https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt')
  if F222_model:
    downloadModel('https://huggingface.co/acheong08/f222/resolve/main/f222.ckpt')
  if DreamShaper_model:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/Dreamshaper_3.32_baked_vae_clip_fix_half.ckpt')
  if OpenJourney_model:
    downloadModel('https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt')
  if Anything_v3_model:
    downloadModel('https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors')
  if Inkpunk_Diffusion_model:
    downloadModel('https://huggingface.co/Envvi/Inkpunk-Diffusion/resolve/main/Inkpunk-Diffusion-v2.ckpt')
  if instruct_pix2pix_model:
    downloadModel('https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.ckpt')

  if v2_1_768_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

  if v2_1_512_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/v2-1_512-ema-pruned.yaml

  if v2_depth_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/512-depth-ema.yaml

  if Model_from_URL:
      for m in Model_from_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Stable-diffusion
        downloadModel(m)
        if Save_a_copy_in_Google_Drive and gMode == GDriveSaveMode.Models_only:
          %cd {output_path}/models
          downloadModel(m)

  # download VAEs
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

def installxformers():
  #!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  #%pip install --no-deps -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl
  %pip install -q xformers

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print('Already python 3.10. Skip install.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def initSaveGoogleDriveModelOnly():
  # Use config file in google drive
  if not os.path.exists(output_path + '/ui-config.json'):
    print("Create new ui-config.json file.")
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'):
    print("Create new config.json file.")
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/config.json -O {output_path + '/config.json'}

  #!rm /content/stable-diffusion-webui/ui-config.json
  !ln -s {output_path + '/ui-config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path}/outputs

  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all models in the models folder
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  models_in_google_drive = glob.glob(output_path + '/models/*')
  print('Models in Google Drive: %s'%models_in_google_drive)
  for f in models_in_google_drive:
    !ln -s {f}

  # link all upscalers in the model folder
  !mkdir -p {root}/stable-diffusion-webui/models/ESRGAN
  %cd {root}/stable-diffusion-webui/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  print('Upscalers in Google Drive: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/stable-diffusion-webui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/stable-diffusion-webui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in google rive
  !mkdir -p {output_path}/VAE
  %cd {root}/stable-diffusion-webui/models
  !rm -rf VAE
  !ln -s {output_path}/VAE



def installControlNet():
  print("Installing ControlNet extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Mikubill/sd-webui-controlnet.git
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet
  !pip install -r requirements.txt

  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')

  print("ControlNet install completed.")

def installDeforum():
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {root}/stable-diffusion-webui/extensions/deforum
  #!cd {root}/stable-diffusion-webui/extensions/deforum; git checkout c42834645805e0f26172888b29f5a9210063db14

def installRegionalPrompter():
  !git clone https://github.com/hako-mikan/sd-webui-regional-prompter {root}/stable-diffusion-webui/extensions/sd-webui-regional-prompter


def installUltimateSDUpscale():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111

def installOpenPoseEditor():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/fkunn1326/openpose-editor


def applyGitFilemode():
  # git default file mode prevents checkout and fail
  print('Apply git filemode false')
  !cd {root}/stable-diffusion-webui/repositories/k-diffusion;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/taming-transformers;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/CodeFormer;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/BLIP;git config core.filemode false

def cloneRepositories():
  !git clone https://github.com/crowsonkb/k-diffusion.git {root}/stable-diffusion-webui/repositories/k-diffusion
  !git clone https://github.com/Stability-AI/stablediffusion.git {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai
  !git clone https://github.com/CompVis/taming-transformers.git {root}/stable-diffusion-webui/repositories/taming-transformers
  !git clone https://github.com/sczhou/CodeFormer.git {root}/stable-diffusion-webui/repositories/CodeFormer
  !git clone https://github.com/salesforce/BLIP.git {root}/stable-diffusion-webui/repositories/BLIP


def installExtensionsFromURL(urls):
  %cd {root}/stable-diffusion-webui/extensions
  for url in urls.split(','):
    !git clone {url}

def lowRamPatch():
  !sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {root}/stable-diffusion-webui/launch.py

def deleteRepos():
  # delete repository directories in webui
  !rm -rf {root}/stable-diffusion-webui/repositories

updatePython()

!mkdir -p {root}
os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui



if gMode == GDriveSaveMode.Everything:
  # delete existing repositories and reclone so the file mode fix can be applied
  # otherwise some will only be cloned in the final launch, causing some to fail to checkout.
  deleteRepos()
  cloneRepositories()
  applyGitFilemode()

# fix torch, torchvision version mismatch error
# !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

# A1111 first launch
%cd {root}/stable-diffusion-webui
# !git checkout -f a9fed7c364061ae6efb37f797b6b522cb3cf7aa2
!git checkout -f 394ffa7b0a7fff3ec484bcd084e673a8b301ccc8
!COMMANDLINE_ARGS="--exit"  python launch.py


if gMode == GDriveSaveMode.Models_only:
  initSaveGoogleDriveModelOnly()

download_models()

if ControlNet:
  installControlNet()

if Deforum:
  installDeforum()

if Regional_Prompter:
  installRegionalPrompter()

if Ultimate_SD_Upscale:
  installUltimateSDUpscale()

if Openpose_Editor:
  installOpenPoseEditor()

installExtensionsFromURL(Extensions_from_URL)

# clear output
if Clear_Log:
  clear()

# if  gMode != GDriveSaveMode.Everything:
#   lowRamPatch()

%cd {root}/stable-diffusion-webui
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers --api'

if NGROK:
  args += f' --ngrok {NGROK} '
else:
  args += ' --share '
args+= ' '+Extra_arguments
print(f'WEBUI ARGUMENTS: {args}')


!python {root}/stable-diffusion-webui/launch.py {args}

/content/drive/MyDrive/AI_PICS/stable-diffusion-webui
WEBUI ARGUMENTS: --gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers --api --ngrok https://9621-24-133-245-165.ngrok-free.app  
Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
Version: v1.4.0
Commit hash: 394ffa7b0a7fff3ec484bcd084e673a8b301ccc8
Installing xformers
Installing ngrok
Installing requirements
Traceback (most recent call last):
  File "/content/drive/MyDrive/AI_PICS/stable-diffusion-webui/launch.py", line 38, in <module>
    main()
  File "/content/drive/MyDrive/AI_PICS/stable-diffusion-webui/launch.py", line 29, in main
    prepare_environment()
  File "/content/drive/MyDrive/AI_PICS/stable-diffusion-webui/modules/launch_utils.py", line 309, in prepare_environment
    run_pip(f"install -r \"{requirements_file}\"", "requirements")
  File "/content/drive/MyDrive/AI_PICS/stable-diffusion-webui/modules/launch_utils.py", line 130, in run_pip
    return run(f'"{python}" -m